In [1]:
import boto.ec2
import boto3

In [2]:
%run aws_helper.py

In [3]:
si = SpotInstances()
ctq = ClusterTaskQueue()

In [4]:
sqs = boto3.resource('sqs')  

In [5]:
new_member_queue = sqs.get_queue_by_name(QueueName='cluster_new_member')
task_commitment_queue = sqs.get_queue_by_name(QueueName='cluster_task_commitment')
task_queue = sqs.get_queue_by_name(QueueName='cluster_task_queue')
# instance metadata: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-instance-metadata.html
# ipython setup: http://ipython.org/ipython-doc/dev/notebook/public_server.html#running-a-public-notebook-server

In [8]:
si.request(num_instances=50, subnet='a')

The following script will be run on startup
#!/bin/bash

echo "==========================="
echo "Starting service 'worker-server'."
initctl status worker-server
echo "==========================="



In [14]:
reqs = si.conn.get_all_spot_instance_requests()

In [15]:
len(reqs)

151

In [17]:
[req.instance_id for req in reqs]

[u'i-251b49fe',
 u'i-000a58db',
 u'i-0a0b59d1',
 u'i-0e0b59d5',
 u'i-49085a92',
 u'i-170a58cc',
 u'i-a40b597f',
 u'i-b40b596f',
 u'i-180a58c3',
 u'i-b00b596b',
 u'i-b2085a69',
 u'i-a50b597e',
 u'i-360b59ed',
 u'i-0b0b59d0',
 u'i-130a58c8',
 u'i-9a0b5941',
 u'i-1c0a58c7',
 u'i-ae0b5975',
 u'i-a90b5972',
 u'i-bb0b5960',
 u'i-0c0a58d7',
 u'i-980b5943',
 u'i-0b0a58d0',
 u'i-61095bba',
 u'i-1b0a58c0',
 u'i-56085a8d',
 u'i-080b59d3',
 u'i-1e0a58c5',
 u'i-69095bb2',
 u'i-b0085a6b',
 u'i-8b085a50',
 u'i-120a58c9',
 u'i-5d085a86',
 u'i-500b598b',
 u'i-110a58ca',
 u'i-990b5942',
 u'i-1f0a58c4',
 u'i-3f0b59e4',
 u'i-b30b5968',
 u'i-330b59e8',
 u'i-62095bb9',
 u'i-ab0b5970',
 u'i-040a58df',
 u'i-b6085a6d',
 u'i-240a58ff',
 u'i-59085a82',
 u'i-6a095bb1',
 u'i-b5085a6e',
 u'i-8f085a54',
 u'i-a20b5979',
 u'i-0c0b59d7',
 u'i-5cb74398',
 u'i-5fb7439b',
 u'i-5ab7439e',
 u'i-fcb04438',
 u'i-68b044ac',
 u'i-cdb74309',
 u'i-81b74345',
 u'i-d9b0441d',
 u'i-87b74343',
 u'i-dcb04418',
 u'i-6db044a9',
 u'i-57b

In [34]:
reservations = si.conn.get_all_reservations(instance_ids = [req.instance_id for req in reqs[0:]])

In [35]:
instances = [instance.public_dns_name for res in reservations for instance in res.instances]
#for res in reservations:
#    for instance in res.instances:
#        print(instance.public_dns_name)

In [20]:
instances[0:5]

[u'ec2-52-89-6-110.us-west-2.compute.amazonaws.com',
 u'ec2-52-89-172-242.us-west-2.compute.amazonaws.com',
 u'ec2-54-148-156-123.us-west-2.compute.amazonaws.com',
 u'ec2-52-26-21-38.us-west-2.compute.amazonaws.com',
 u'ec2-52-88-167-229.us-west-2.compute.amazonaws.com']

In [ ]:
si.print_public_dns_names()

In [ ]:
ctq.add_task('hello yejllow 17:27')

In [ ]:
ctq.add_task('git:pull')

In [ ]:
#ctq.add_task('parse:01')
ctq.add_task('parse:test/008:5644:10100')

In [ ]:
ctq.add_task('XXXparse:tmp/01:404075:405000')

In [37]:
run_id = 'prod/08'
total_ids = 151
for i in range(total_ids):
    #ctq.add_task('quit')
    #ctq.add_task('git:pull')
    ctq.add_task('parse:%s:%d:%d' % (run_id, i, total_ids))
    #ctq.add_task('parse:%s' % (run_id))

In [ ]:
ctq.add_task('quit')

In [21]:
started = []

In [29]:
for i in range(5):
    msgs = new_member_queue.receive_messages(MessageAttributeNames=['Worker'])
    for msg in msgs:
        worker_dns_name = 'unknown'
        if msg.message_attributes is not None:
            worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
        print('%s: %s' % (worker_dns_name, msg.body))
        msg.delete()
        started.append(worker_dns_name)

ec2-54-68-252-179.us-west-2.compute.amazonaws.com: new_member


In [30]:
len(started)

151

In [ ]:
'ec2-52-27-130-165.us-west-2.compute.amazonaws.com' in started

In [36]:
list(set(instances) - set(started)), list(set(started) - set(instances))

([], [])

In [ ]:
res = []

In [ ]:
for i in range(5):
    msgs = task_commitment_queue.receive_messages(MessageAttributeNames=['Worker'])
    for msg in msgs:
        worker_dns_name = 'unknown'
        if msg.message_attributes is not None:
            worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
        print('%s: %s' % (worker_dns_name, msg.body))
        res.append((worker_dns_name, msg.body))
        msg.delete()

In [ ]:
#res.append(('ec2-54-68-173-118.us-west-2.compute.amazonaws.com','parse:prod/07:3:200' ))

In [ ]:
len(res)

In [ ]:
len(set([a[0] for a in accepted]))

In [ ]:
workers = [r[0] for r in res]

In [ ]:
set(hosts) - set(workers)

In [ ]:
#Delete 'quit' message after the end
msgs = task_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    #msg.delete()

In [ ]:
new_member_queue.purge()
task_queue.purge()
task_commitment_queue.purge()

In [ ]:
hosts = [
    "ec2-52-89-193-15.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-78.us-west-2.compute.amazonaws.com",
    "ec2-54-68-193-159.us-west-2.compute.amazonaws.com",
    "ec2-52-88-201-154.us-west-2.compute.amazonaws.com",
    "ec2-52-89-15-13.us-west-2.compute.amazonaws.com",
    "ec2-52-89-45-252.us-west-2.compute.amazonaws.com",
    "ec2-54-69-129-216.us-west-2.compute.amazonaws.com",
    "ec2-52-89-235-49.us-west-2.compute.amazonaws.com",
    "ec2-54-68-201-203.us-west-2.compute.amazonaws.com",
    "ec2-52-10-43-242.us-west-2.compute.amazonaws.com",
    "ec2-52-88-239-14.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-4.us-west-2.compute.amazonaws.com",
    "ec2-52-89-133-116.us-west-2.compute.amazonaws.com",
    "ec2-54-68-203-141.us-west-2.compute.amazonaws.com",
    "ec2-54-69-128-40.us-west-2.compute.amazonaws.com",
    "ec2-52-89-243-103.us-west-2.compute.amazonaws.com",
    "ec2-54-68-167-111.us-west-2.compute.amazonaws.com",
    "ec2-54-68-149-49.us-west-2.compute.amazonaws.com",
    "ec2-52-88-140-31.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-17.us-west-2.compute.amazonaws.com",
    "ec2-54-68-108-253.us-west-2.compute.amazonaws.com",
    "ec2-52-89-135-95.us-west-2.compute.amazonaws.com",
    "ec2-52-89-221-244.us-west-2.compute.amazonaws.com",
    "ec2-52-89-254-95.us-west-2.compute.amazonaws.com",
    "ec2-54-68-166-47.us-west-2.compute.amazonaws.com",
    "ec2-54-69-43-170.us-west-2.compute.amazonaws.com",
    "ec2-52-89-151-225.us-west-2.compute.amazonaws.com",
    "ec2-54-69-133-194.us-west-2.compute.amazonaws.com",
    "ec2-52-26-206-14.us-west-2.compute.amazonaws.com",
    "ec2-52-88-57-125.us-west-2.compute.amazonaws.com",
    "ec2-54-69-102-121.us-west-2.compute.amazonaws.com",
    "ec2-52-88-5-19.us-west-2.compute.amazonaws.com",
    "ec2-52-25-5-100.us-west-2.compute.amazonaws.com",
    "ec2-54-68-125-99.us-west-2.compute.amazonaws.com",
    "ec2-52-89-55-24.us-west-2.compute.amazonaws.com",
    "ec2-54-68-179-101.us-west-2.compute.amazonaws.com",
    "ec2-54-69-7-70.us-west-2.compute.amazonaws.com",
    "ec2-54-68-46-25.us-west-2.compute.amazonaws.com",
    "ec2-52-27-101-119.us-west-2.compute.amazonaws.com",
    "ec2-54-69-16-198.us-west-2.compute.amazonaws.com",
    "ec2-52-27-202-38.us-west-2.compute.amazonaws.com",
    "ec2-52-89-241-245.us-west-2.compute.amazonaws.com",
    "ec2-52-89-188-228.us-west-2.compute.amazonaws.com",
    "ec2-54-68-163-25.us-west-2.compute.amazonaws.com",
    "ec2-54-69-100-163.us-west-2.compute.amazonaws.com",
    "ec2-52-89-215-11.us-west-2.compute.amazonaws.com",
    "ec2-52-89-93-129.us-west-2.compute.amazonaws.com",
    "ec2-54-68-148-244.us-west-2.compute.amazonaws.com",
    "ec2-54-68-205-216.us-west-2.compute.amazonaws.com",
    "ec2-54-69-111-80.us-west-2.compute.amazonaws.com",
    "ec2-54-69-16-189.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-59.us-west-2.compute.amazonaws.com",
    "ec2-52-89-248-201.us-west-2.compute.amazonaws.com",
    "ec2-52-88-167-15.us-west-2.compute.amazonaws.com",
    "ec2-54-68-102-238.us-west-2.compute.amazonaws.com",
    "ec2-54-68-197-197.us-west-2.compute.amazonaws.com",
    "ec2-52-24-252-202.us-west-2.compute.amazonaws.com",
    "ec2-52-26-56-255.us-west-2.compute.amazonaws.com",
    "ec2-52-11-188-93.us-west-2.compute.amazonaws.com",
    "ec2-52-89-37-225.us-west-2.compute.amazonaws.com",
    "ec2-54-69-134-170.us-west-2.compute.amazonaws.com",
    "ec2-54-68-230-132.us-west-2.compute.amazonaws.com",
    "ec2-52-26-247-58.us-west-2.compute.amazonaws.com",
    "ec2-54-69-22-62.us-west-2.compute.amazonaws.com",
    "ec2-52-88-148-10.us-west-2.compute.amazonaws.com",
    "ec2-54-68-104-52.us-west-2.compute.amazonaws.com",
    "ec2-54-68-115-107.us-west-2.compute.amazonaws.com",
    "ec2-52-89-241-191.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-58.us-west-2.compute.amazonaws.com",
    "ec2-54-69-11-201.us-west-2.compute.amazonaws.com",
    "ec2-52-10-114-136.us-west-2.compute.amazonaws.com",
    "ec2-54-68-202-216.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-34.us-west-2.compute.amazonaws.com",
    "ec2-52-88-34-128.us-west-2.compute.amazonaws.com",
    "ec2-52-88-176-236.us-west-2.compute.amazonaws.com",
    "ec2-52-89-208-176.us-west-2.compute.amazonaws.com",
    "ec2-52-24-102-152.us-west-2.compute.amazonaws.com",
    "ec2-52-89-227-161.us-west-2.compute.amazonaws.com",
    "ec2-54-68-203-253.us-west-2.compute.amazonaws.com",
    "ec2-52-89-90-171.us-west-2.compute.amazonaws.com",
    "ec2-52-89-196-70.us-west-2.compute.amazonaws.com",
    "ec2-52-88-12-15.us-west-2.compute.amazonaws.com",
    "ec2-52-89-175-90.us-west-2.compute.amazonaws.com",
    "ec2-52-89-230-26.us-west-2.compute.amazonaws.com",
    "ec2-54-68-156-176.us-west-2.compute.amazonaws.com",
    "ec2-52-89-163-39.us-west-2.compute.amazonaws.com",
    "ec2-54-69-112-242.us-west-2.compute.amazonaws.com",
    "ec2-52-25-123-5.us-west-2.compute.amazonaws.com",
    "ec2-52-88-216-208.us-west-2.compute.amazonaws.com",
    "ec2-52-25-242-153.us-west-2.compute.amazonaws.com",
    "ec2-54-68-131-38.us-west-2.compute.amazonaws.com",
    "ec2-52-88-135-209.us-west-2.compute.amazonaws.com",
    "ec2-52-27-6-21.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-66.us-west-2.compute.amazonaws.com",
    "ec2-52-89-222-49.us-west-2.compute.amazonaws.com",
    "ec2-52-89-226-215.us-west-2.compute.amazonaws.com",
    "ec2-52-89-187-92.us-west-2.compute.amazonaws.com",
    "ec2-52-24-99-129.us-west-2.compute.amazonaws.com",
    "ec2-52-89-132-66.us-west-2.compute.amazonaws.com",
    "ec2-52-89-73-242.us-west-2.compute.amazonaws.com"   
    ]
